# Comparativa de modelos de clasificación de texto

Iniciamos adecuando el ambiente de ejcución, instalando e importando los paquetes necesarios para el análisis

In [1]:
%%capture --no-display
!pip install keras
!pip install tensorflow
!pip install xgboost
!pip install transformers

In [2]:
from keras.models import Sequential
from keras import layers
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
import seaborn as sn
import numpy as np
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.callbacks import EarlyStopping
import joblib

In [3]:
tf.keras.backend.set_image_data_format("channels_last")

Llamamos del notebook que construye el dataset vectorizado mediante TF-IDF y genera nuestros conjuntos de entrenamiento y validación.

In [4]:
%run 1_MulticategoricalTFIDF.ipynb

[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1125)>
[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1125)>


showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


<ipython-input-4-3fec7da56bed>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_facturas['Categoria'] = df_facturas['issuer'].map(diccionario)
<ipython-input-4-3fec7da56bed>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_facturas['Categoria'] = "No factura"


sfarma are drogas fragua nit sparma droguerias cipriano calle san cipriano tel aut numeracion fac regimen comun tarifa tarifa regimen regimen comun obs regimen grandes contribuyentes fecha emision feoha vigencia prefilo consecutivos factura venta mostrador fecha hora caja ciudad bogota vendedor daniela alejandra carreno castellano nombre cliente primer apellido generico segundo apellido kkkkkkk cedula nit direccion calle telefono producto cant pvp dtd imp valor buprofend metocarbamol tabletas pres und levotironina tes removedor lander vitaminae mad schickquattro uds pres fraccion und paleta tosh pasion palet apolet surtica betametasona cpema nitazoxanda tbs crema corega sulfato magnesia drogam melterios pastillas masticables sbs pres sobri und noraver grpabebidanoche sbs vich vaporub ups pres fraccion unid flumuc age mes sbs pres fraccion und pax dia naranja ses pres fraccion xiund tapaboca termosellado biokemical soul pres fraccion lund paletaje surtida labinpina tabletas pres und boo

Dado que la variable de respuesta cuenta con diferentes categorías tipo "string", a continuación se codifica la variable a una matriz numérica binaria para poder entrenar las redes neuronales más adelante. Este proceso no es necesario para los demas modelos (SVM, Random Forest, XGBoost)

In [5]:
y_train_d = pd.get_dummies(y_train)
y_test_d = pd.get_dummies(y_test)

## Definición de modelos de clasificación de texto

Para este proyecto se utilizarán los siguientes modelos, cuyos parámetros fueron ajustados y seleccionados mediante un grid search con validación cruzada:

* **Linear Support Vector Classifier:**
  * Parámetro de regularización: 10
  * Función de pérdida: squared hinge
  * Máximo número de iteraciones: 1.000
  * Algoritmo para resolver el problema de optimización: Dual
  * Estrategia multiclase: ovr, es decir que entrena n-clases uno contra el resto
  * Norma utilizada en la penalización: 'l2', es decir el estándar utilizado en SVC
<br>
<br>
* **XGBoost Classifier:**
  * Tipo de refuerzo "Booster": gbtree, utiliza modelos basados en árboles de decisión
  * Tasa de aprendizaje: 0.3
  * Máxima profundidad de los árboles de decisión: 6
<br>
<br>
* **Random Forest Classifier:**
  * Función para medir la calidad de la división en los arboles: Entropía
  * Máxima profundidad de los árboles de decisión: 20
  * Máximo número de características a tener en cuenta al buscar la mejor división en los árboles: sqrt, es decir que max_features = sqrt(n_features)
  * Máximo de nodos hoja: 1.000
  * Número mínimo de muestras requeridas para estar en un nodo de hoja: 10
  * Número de árboles: 10.000
<br>
<br>
* **Perceptrón multicapa:**
  * Estructura:
      * Red neuronal densamente conectada
      * Una capa oculta con 256 neuronas y función de actuvación "relu"
      * Capa de salida con 5 neuronas y función de activación "softmax"
  * Optimización:
      * Función de pérdida: Crossentropía binaria
      * Optimizador: ADAM
      * Épocas: 10
      * Tamaño de los lotes: 64
<br>
<br>
* **Red Neuronal LSTM:**
  * Estructura:
      * Red neuronal recurrente LSTM
      * Capa de embedding con dimensión de 100, máximo de 50.000 palabras, y secuencia máxima de 250 caracteres
      * Capa de Long short-term memory con 256 unidades ocultas, un dropout y dropout del estado recurrente del 20%
      * Una capa oculta densamente conectada con 128 neuronas y función de actuvación "relu"
      * Capa de salida densamente conectada con 5 neuronas y función de activación "softmax"
  * Optimización:
      * Función de pérdida: Crossentropía binaria
      * Optimizador: ADAM
      * Épocas: 50
      * Tamaño de los lotes: 128
<br>
<br>

In [6]:
def svc_clf(X_train,X_test,y_train,y_test):
    clf = LinearSVC(C=10, loss='squared_hinge', max_iter= 1000,
                    dual = True, multi_class= 'ovr',
                    penalty= 'l2', random_state= 42)
    clf.fit(X_train, y_train)
    preds = clf.predict(X_test)
    accuracy = accuracy_score (y_test, preds)
    return accuracy,preds


def xgb_clf(X_train,X_test,y_train,y_test):
    clf = xgb.XGBClassifier(booster = 'gbtree', 
                            eta = 0.3, max_depth = 6)
    
    clf.fit(X_train, y_train)
    preds = clf.predict(X_test)
    accuracy = accuracy_score (y_test, preds)
    return accuracy,preds


def randf_clf(X_train,X_test,y_train,y_test):
    clf = RandomForestClassifier(
        criterion = 'entropy', max_depth = 20,
        max_features = 'sqrt', max_leaf_nodes = 1000,
        min_samples_leaf = 10, n_estimators = 10000)
    
    clf.fit(X_train, y_train)
    preds = clf.predict(X_test)
    accuracy = accuracy_score (y_test, preds)
    return accuracy,preds


def perceptron(X_train, X_test, y_train, y_test):
    input_dim = X_train.shape[1]  # Number of features

    model = Sequential()
    model.add(layers.Dense(256, input_dim=input_dim, activation='relu'))
    model.add(layers.Dense(5, activation='softmax'))

    model.compile(loss='binary_crossentropy', 
                  optimizer='adam', 
                  metrics=['accuracy'])

    history = model.fit(X_train, y_train,
                        epochs=50,
                        verbose=False,
                        validation_data=(X_test, y_test),
                        batch_size=128)

    loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
    preds = model.predict(X_test)
    preds = (preds > 0.5).astype("int32")
    return accuracy,preds



def lstm(dataframe):
    
    MAX_NB_WORDS = 50000
    MAX_SEQUENCE_LENGTH = 250
    EMBEDDING_DIM = 100
    tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
    tokenizer.fit_on_texts(dataframe['text'].values)
    word_index = tokenizer.word_index
    
    X = tokenizer.texts_to_sequences(dataframe['text'].values)
    X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
    Y = pd.get_dummies(dataframe['Categoria']).values
    
    X_train_lstm, X_test_lstm, Y_train_lstm, Y_test_lstm = train_test_split(X,Y, test_size = 0.30, stratify = dataframe.Categoria, random_state = 42)

    model = Sequential()
    model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
    model.add(LSTM(256, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(5, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    epochs = 10
    batch_size = 64

    history = model.fit(X_train_lstm, Y_train_lstm, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

    accuracy = model.evaluate(X_test_lstm,Y_test_lstm)
    preds = model.predict(X_test_lstm)
    preds = (preds > 0.5).astype("int32")
    
    return accuracy,preds,Y_test_lstm



def acc_promedio(n, modelo):
    suma_acuracia = 0
    for i in range(n):
        suma_acuracia += modelo[0]
    return  suma_acuracia/n

## Entrenamiento de los modelos

A continuación entrenaremos los modelos previamente definidos, y calcularemos el tiempo de entrenamiento con el fin de incluir este indicador como variable de desempeño a comparar para la selección del mejor algoritmo de clasificación.

In [7]:
%%time
acc_svc, pred_svc = svc_clf(X_train,X_test,y_train,y_test)
print(acc_svc)

0.9630390143737166
CPU times: user 263 ms, sys: 24.5 ms, total: 287 ms
Wall time: 251 ms


In [8]:
%%time
acc_xgb, pred_xgb = xgb_clf(X_train,X_test,y_train,y_test)
print(acc_xgb)

/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[23:27:01] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0.9589322381930184
CPU times: user 16min 40s, sys: 37.5 s, total: 17min 18s
Wall time: 1min 42s


In [9]:
%%time
acc_rfc, pred_rfc = randf_clf(X_train,X_test,y_train,y_test)
print(acc_rfc)

0.8562628336755647
CPU times: user 1min 53s, sys: 599 ms, total: 1min 54s
Wall time: 1min 54s


In [10]:
%%time
acc_perc, pred_perc = perceptron(X_train,X_test,y_train_d,y_test_d)
print(acc_perc)

0.9650924205780029
CPU times: user 1min 32s, sys: 37 s, total: 2min 9s
Wall time: 14.3 s


In [11]:
%%time
acc_lstm, pred_lstm, Y_test_lstm = lstm(dataframe)
print(acc_lstm)

Epoch 1/10
16/16 [==============================] - 17s 972ms/step - loss: 1.4727 - accuracy: 0.3855 - val_loss: 1.3758 - val_accuracy: 0.5263
Epoch 2/10
16/16 [==============================] - 15s 970ms/step - loss: 1.1789 - accuracy: 0.6047 - val_loss: 1.0808 - val_accuracy: 0.4474
Epoch 3/10
16/16 [==============================] - 24s 2s/step - loss: 0.6419 - accuracy: 0.7769 - val_loss: 0.5159 - val_accuracy: 0.8070
Epoch 4/10
16/16 [==============================] - 66s 4s/step - loss: 0.2915 - accuracy: 0.9070 - val_loss: 0.3925 - val_accuracy: 0.8772
Epoch 5/10
16/16 [==============================] - 26s 2s/step - loss: 0.1569 - accuracy: 0.9432 - val_loss: 0.3110 - val_accuracy: 0.9123
Epoch 6/10
16/16 [==============================] - 27s 2s/step - loss: 0.0903 - accuracy: 0.9560 - val_loss: 0.3107 - val_accuracy: 0.8860
Epoch 7/10
16/16 [==============================] - 40s 3s/step - loss: 0.0521 - accuracy: 0.9795 - val_loss: 0.2601 - val_accuracy: 0.9123
Epoch 8/10
16/

Como se puede observar, el modelo con menor tiempo de entrenamiento es el Linear Support Vector Classifier, seguido por el percptrón multicapa. Por el contrario, la red neuronal LSTM es la que cuenta con un mayor tiempo de entrenamiento. 

## Comparativo de desempeño

In [12]:
%%capture --no-stdout

print("Informe de desempeño SVC:")
print(classification_report(y_test, pred_svc))

print("Informe de desempeño XGB:")
print(classification_report(y_test, pred_xgb))

print("Informe de desempeño Random Forest:")
print(classification_report(y_test, pred_rfc))

print("Informe de desempeño Perceptrón Multicapa:")
print(classification_report(y_test_d, pred_perc))

print("Informe de desempeño LSTM:")
print(classification_report(Y_test_lstm, pred_lstm))

Informe de desempeño SVC:
                   precision    recall  f1-score   support

     Alimentación       0.95      0.98      0.97       170
Grande superficie       0.95      0.97      0.96        99
             Moda       0.93      0.78      0.85        18
       No factura       0.99      0.98      0.99       158
Salud y Bienestar       0.93      0.90      0.92        42

         accuracy                           0.96       487
        macro avg       0.95      0.92      0.94       487
     weighted avg       0.96      0.96      0.96       487

Informe de desempeño XGB:
                   precision    recall  f1-score   support

     Alimentación       0.93      0.99      0.96       170
Grande superficie       0.94      0.97      0.96        99
             Moda       0.93      0.72      0.81        18
       No factura       1.00      0.98      0.99       158
Salud y Bienestar       0.97      0.81      0.88        42

         accuracy                           0.96       487

Luego de analizar la evaluación de desmpeño de cada modelo, encontramos que los más eficientes y al mismo tiempo, los que mejor se ajustan a los datos, sin caer aparentemente en sobreajuste son el Linear Support Vector Classifier y el Perceptrón Multicapa.

En efecto, tanto el LinearSVC como el perceptrón multicapa muestran unas métricas F1-Score por encima del 90% para todas las categorías, menos para "moda", para la cual registran un desempeño del 85%, a pesar de contar con un bajo número de registros para entrenar el modelo. 

En cuanto al XGBoost, se observan métricas bastante similares, por encima del 90% para todas las categorías, sin embargo, para "salud y bienestar" y "moda" el desempeño se reduce para ubicarse en 88% y 81%. respectivamente.

Para el caso de la red LSTM se observa unas métricas F1-Score por encima del 80% para todas las categorías, menos para "salud y bienestar" y "moda", para los cuales se registran un desempeño de 79% y 50%, respectivamente. Lo anterior se puede explicar por el bajo número de registros para entrenar el modelo en dichas categorías, lo cual se ve profundizado teniendo en cuenta que este tipo de redes requieren de un gran número de registros para entrenarse adecuadamente.

Finalmente, el Random Forest evidencia que el algortimo no logran ajustarse del todo a los datos, en especial para la categoría "moda", la cual no logra ser identificada en ninguna de las observaciones de evaluación. Como resultado se observa el Accuracy global más bajo entre todos los modelos.

De esta manera, se selecciona en primer lugar al Linear Support Vector Classifier dado que es el que cuenta con mejores métricas y al mismo tiempo un menor tiempo de entrenamiento. En segundo lugar se selecciona al perceptrón multicapa, dado que cuenta con métricas igual de satisfactorias al LinearSVC, aunque con un tiempo de entreno mayor. Finalmente se descartan los demás modelos dado que muestran peores métricas, al mismo tiempo en que requieren un mayor tiempo de entrenamiento. 

### Guardar los modelos entrenados

Ahora que hemos seleccionado los modelos a utilizar para poner a prueba el algoritmo final de clasificación, guardaremos los modelos entrenados con el set de datos de entrenamiento, para aplicarlos dentro del algoritmo final.

In [13]:
clf = LinearSVC(C=10, loss='squared_hinge', max_iter= 1000,
                dual = True, multi_class= 'ovr',
                penalty= 'l2', random_state= 42)
clf.fit(X_train, y_train)

filename = "SVM.joblib"
joblib.dump(clf, filename)

['SVM.joblib']

In [14]:
input_dim = X_train.shape[1]

model = Sequential()
model.add(layers.Dense(256, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(5, activation='softmax'))

model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

history= model.fit(X_train, y_train_d,
                   epochs=50,
                   verbose=False,
                   batch_size=128)

model.save("Perceptron.h5")

### Ensamble

In [15]:
from tensorflow import keras

In [16]:
def ensamble (X): 
    
    modelo_svm = joblib.load("SVM.joblib")
    pred_svm = modelo_svm.predict(X)

    modelo_perceptron = keras.models.load_model("Perceptron.h5")
    pred_perceptron = modelo_perceptron.predict(X)
    pred_perceptron = (pred_perceptron >= 0.5).astype("int32")
    
    pred_perc = []
    lst = ['Alimentación', 'Grande superficie', 'Moda', 'No factura', 'Salud y Bienestar']  
    for i in pred_perceptron:
        maxindex = np.argmax(i)
        pred = lst[maxindex]
        pred_perc.append(pred)
    
    pred=[]
    for i in range(len(pred_svm)):
        if pred_svm[i] == 'No factura':
            pred.append(pred_svm[i])
        else:
            pred.append(pred_perc[i])
        
    return pred

In [17]:
pred_ensamble = ensamble(X_test)

In [18]:
modelo_svm = joblib.load("SVM.joblib")
pred_svm = modelo_svm.predict(X_test)

In [19]:
modelo_perceptron = keras.models.load_model("Perceptron.h5")
pred_perceptron = modelo_perceptron.predict(X_test)
pred_perceptron = (pred_perceptron >= 0.5).astype("int32")

In [20]:
print("Informe de desempeño Ensamble:")
print(classification_report(y_test, pred_ensamble))

print("Informe de desempeño SVC:")
print(classification_report(y_test, pred_svm))

print("Informe de desempeño Perceptrón:")
print(classification_report(y_test_d, pred_perceptron))

Informe de desempeño Ensamble:
                   precision    recall  f1-score   support

     Alimentación       0.95      0.98      0.97       170
Grande superficie       0.95      0.97      0.96        99
             Moda       0.93      0.78      0.85        18
       No factura       0.97      0.98      0.98       158
Salud y Bienestar       0.97      0.86      0.91        42

         accuracy                           0.96       487
        macro avg       0.96      0.91      0.93       487
     weighted avg       0.96      0.96      0.96       487

Informe de desempeño SVC:
                   precision    recall  f1-score   support

     Alimentación       0.95      0.98      0.97       170
Grande superficie       0.95      0.97      0.96        99
             Moda       0.93      0.78      0.85        18
       No factura       0.99      0.98      0.99       158
Salud y Bienestar       0.93      0.90      0.92        42

         accuracy                           0.96     

/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


El ensamble no sirve porque incluso cuando ha pasado el filtro del SVC identificando que es una factura, le da el chance al perceptrón de que se equivoque clasificandola como una no factura